In [1]:
import os
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records
import models
models_path = models.__path__[0]

fix_block_number = None

fix_block_number = 41028518 + 3010

cmf_param = {
    'block_number': fix_block_number,
    'chain_id': 137,
    # 'api_url': 'http://localhost:8700'
}

context, _model_loader = create_cmf(cmf_param, True)

fix_block_number = context.block_number

print(fix_block_number)


Credmark context created with 
- chain_id=137
- block_number=41031528
- chain_provider_url=https://ca...e77/
- model_loader_path=['/home/yangye/dev/credmark/credmark-models-py/models']
- api_url=https://gateway.credmark.com
- use_local_models=None

41031528


In [2]:
Token('0x111111517e4929d3dcbdfa7cce55d30d4b6bc4d6').as_erc20(
    set_loaded=True).symbol


'ICHI'

## 1. Get History

In [7]:
from models.tmp_abi_lookup import ICHI_VAULT, ICHI_VAULT_FACTORY, UNISWAP_V3_POOL_ABI

from models.credmark.protocols.ichi.vault import IchiVaults

vault_rows = []

event_sel = 'Withdraw'
event_sel = 'Deposit'

# burn partial position ?, burn share o
# mint position based on how much token to deposit

for vault_addr in IchiVaults.ICHI_POLYGON_VAULTS:
    # if Address(vault_addr) == Address('0x70Aa3c8e256c859e52c0B8C263f763D9051B95e1'):
    # vault_addr = Address('0x692437de2cAe5addd26CCF6650CaD722d914d974')
    vault_ichi = Token(vault_addr).set_abi(abi=ICHI_VAULT, set_loaded=True)

    dfs = {}

    for evt in vault_ichi.abi.events:
        if evt in ['Rebalance', 'Deposit', 'Withdraw']:
            df = pd.DataFrame(vault_ichi.fetch_events(
                vault_ichi.events[evt], from_block=25697834))  # , by_range=10000
            dfs[evt] = df

    print((vault_addr, dfs['Deposit'].shape,
          dfs['Withdraw'].shape, dfs['Rebalance'].shape))

    for n_row, row in dfs[event_sel].iterrows():
        amount0 = row['amount0']
        amount1 = row['amount1']
        shares = row['shares']
        block_number = row['blockNumber']
        vault_info = context.run_model(
            'ichi.vault-info', {"address": vault_addr}, block_number=block_number)
        if vault_info['allowed_token'] == 1:
            vault_rows.append((n_row, block_number, vault_addr, 'token1',
                              vault_info['vault_token_ratio'], amount1 / shares, vault_info['vault_token_ratio'] - amount1 / shares))
        else:
            vault_rows.append((n_row, block_number, vault_addr, 'token0',
                              vault_info['vault_token_ratio'], amount0 / shares, vault_info['vault_token_ratio'] - amount0 / shares))

df_vault_rows = pd.DataFrame(vault_rows, columns=[
                             'n_row', 'block_number', 'vault_addr', 'token_n', 'vault_token_ratio', 'amount_per_share', 'diff_ratio'])
display(df_vault_rows)
df_vault_rows.to_csv(f'vault_rows2_{event_sel}.csv', index=False)


('0x9ff3C1390300918B40714fD464A39699dDd9Fe00', (15, 13), (2, 13), (116, 14))
('0x692437de2cAe5addd26CCF6650CaD722d914d974', (9, 13), (1, 13), (96, 14))
('0x70Aa3c8e256c859e52c0B8C263f763D9051B95e1', (5, 13), (2, 13), (178, 14))
('0xf461063819fFBC6e22704aDe1861B0dF3BaC9585', (6, 13), (0, 0), (81, 14))
('0xf3De925524cE6bBa606107CFCB2A7A6259CD01EA', (8, 13), (2, 13), (151, 14))
('0x711901e4b9136119Fb047ABe8c43D49339f161c3', (32, 13), (25, 13), (116, 14))
('0x3ac9b3db3350A515c702ba19a001d099d4a5F132', (24, 13), (12, 13), (275, 14))
('0xf7B1ab2545451b60345FA3aB8C5210d53c703c98', (6, 13), (0, 0), (59, 14))
('0xB05bE549a570e430e5ddE4A10a0d34cf09a7df21', (7, 13), (4, 13), (111, 14))
('0x74F9d8861D42Ac09759aDE7809A67cF053dc7bA5', (13, 13), (1, 13), (101, 14))
('0xE5bf5D33C617556B91558aAfb7BeadB68E9Cea81', (83, 13), (67, 13), (112, 14))
('0x5a0834EBaFdF97DB54f45a43290b6B09D4226ec6', (4, 13), (0, 0), (53, 14))
('0x5394eb43700Ce8fBbF4CB83E8b52ea73b71426B6', (3, 13), (1, 13), (21, 14))
('0xac6c0264

,n_row,block_number,vault_addr,token_n,vault_token_ratio,amount_per_share,diff_ratio
0,0,39589655,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,token0,1.202725e+03,1.202725e+03,0.000000e+00
1,1,39596382,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,token0,1.207304e+03,1.207304e+03,-1.078907e-04
2,2,39786283,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,token0,1.212089e+03,1.213482e+03,-1.392459e+00
3,3,39786518,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,token0,1.212190e+03,1.213456e+03,-1.265650e+00
4,4,39787107,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,token0,1.212310e+03,1.213403e+03,-1.092861e+00
...,...,...,...,...,...,...,...
274,2,40543827,0xFc089714519E84B7ce0a4023bfEE0D99F6d767dA,token0,6.813946e-12,6.820730e-12,-6.784127e-15
275,0,37678381,0x21e6910A769d10ef4236107493406a9788C758a3,token0,1.407044e+13,1.407044e+13,0.000000e+00
276,1,37734177,0x21e6910A769d10ef4236107493406a9788C758a3,token0,1.407044e+13,1.407044e+13,-1.953125e-03
277,2,37932696,0x21e6910A769d10ef4236107493406a9788C758a3,token0,1.407395e+13,1.407398e+13,-2.682752e+07


from credmark.cmf.engine.model_api import CREDMARK_API_KEY

```bash
time curl -X 'POST' https://gateway.credmark.com/v1/model/run \
-H 'accept: application/json' \
-H 'Content-Type: application/json' \
-H "Authorization: Bearer $CREDMARK_API_KEY" \
-d '{ "slug": "ichi.vaults-performance", "chainId": 137, "blockNumber": "latest", "input": {} }' | jq
````

In [7]:
result = context.run_model('ichi.vaults-performance')
df_vaults_performance = pd.DataFrame(result['data'])
df_vaults_performance.to_csv('df_vaults_performance_updated3.csv')

assert df_vaults_performance.loc[df_vaults_performance.irr_hold_5050 <
                                 df_vaults_performance.irr_uniswap].empty

(df_vaults_performance
    .loc[:, ['vault', 'tvl', 'token0_symbol', 'token1_symbol',
             'allowed_token', 'days_since_first_deposit', 'irr', 'irr_hold_5050', 'irr_uniswap']]
    .sort_values('tvl', ascending=False)
    .reset_index(drop=True))

,vault,tvl,token0_symbol,token1_symbol,allowed_token,days_since_first_deposit,irr,irr_hold_5050,irr_uniswap
0,0x9ff3c1390300918b40714fd464a39699ddd9fe00,256636.750718,WMATIC,WETH,WMATIC,38.282280,0.600940,3.928452,3.395289
1,0x692437de2cae5addd26ccf6650cad722d914d974,185928.544227,LINK,WETH,LINK,38.255174,0.504991,0.631415,0.612045
2,0xf461063819ffbc6e22704ade1861b0df3bac9585,116938.777505,WETH,BAL,BAL,38.274780,0.480364,0.827098,0.794648
3,0xf3de925524ce6bba606107cfcb2a7a6259cd01ea,102512.147142,GHST,WETH,GHST,38.275312,1.973346,13.718038,9.936835
4,0x3ac9b3db3350a515c702ba19a001d099d4a5f132,76979.525853,USDC,WETH,USDC,98.693750,0.139812,1.256760,1.096610
5,0xf7b1ab2545451b60345fa3ab8c5210d53c703c98,54937.921549,CRV,WETH,CRV,19.048727,0.571910,3.917798,3.626062
6,0xb05be549a570e430e5dde4a10a0d34cf09a7df21,54617.151426,USDC,WETH,WETH,19.082836,-0.171541,-0.514907,-0.521756
7,0x74f9d8861d42ac09759ade7809a67cf053dc7ba5,47748.246552,SUSHI,WETH,SUSHI,38.280475,1.399242,5.186648,4.343435
8,0x5a0834ebafdf97db54f45a43290b6b09d4226ec6,45234.869465,ICHI,WETH,ICHI,59.165984,2.122593,1.888061,1.671533
9,0xac6c0264511eeec305da9afc2e1aba08409f99f6,37785.788640,WMATIC,ICHI,ICHI,39.962998,0.088487,-0.460941,-0.472898


In [12]:
vault_info['total_amount_in_token'] * 1e6 / 5.042621652e09
vault_info


{'token0': '0x111111517e4929D3dcbdfa7CCe55d30d4B6BC4d6',
 'token1': '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174',
 'token0_symbol': 'ICHI',
 'token1_symbol': 'USDC',
 'allowed_token': 1,
 'token0_amount': 0.0,
 'token1_amount': 5042.621652,
 'total_amount_in_token': 5042.621652,
 'total_supply': 5.042621652e-09,
 'vault_token_ratio': 1000000000000.0,
 'pool_price0': 0.2814968629836415,
 'token0_price_chainlink': nan,
 'token1_price_chainlink': 1.00045454,
 'vault_token_value_chainlink': nan}

- vault operations
- withdraw, transfer
- deposit, transfer
- rebalance

- 0x8ac3d7cd56816da9fb45e7640aa70a24884e02f7
- 0x0768A75F616B98ee0937673bD83B7aBF142236Ea', '0xED26b64F351A04Acbf16AA3782969e4a53B12c54', 'WETH', 'DPI', False, True, 280.73224104767195

all transfers

In [9]:
(
    vault_ichi.functions.getTotalAmounts().call(),
    vault_ichi.functions.getBasePosition().call(),
    vault_ichi.functions.getLimitPosition().call(),
    token0.balance_of(vault_ichi.address.checksum),
    token1.balance_of(vault_ichi.address.checksum),
)

# 286852599603313080740 - 26182028192636748 - 286826417575120443992
# 274646900430582100 - 274646900430582100 - 0

# totalAmount0	totalAmount1	feeAmount0	feeAmount1	totalSupply
# 274911385738111039	286846445532138665343	16019734579489	0	280732241047671948180

([8620625751508569036, 983616],
 [3882924048014, 8620625751506928060, 583806],
 [15481272409821, 0, 399810],
 1640976,
 0)

In [10]:
# vault_pool()
base_lower = vault_ichi.functions.baseLower().call()
base_upper = vault_ichi.functions.baseUpper().call()

limit_lower = vault_ichi.functions.limitLower().call()
limit_upper = vault_ichi.functions.limitUpper().call()

base_position_key = context.web3.solidityKeccak(['address', 'int24', 'int24'], [
                                                vault_ichi.address.checksum, base_lower, base_upper])
limit_position_key = context.web3.solidityKeccak(['address', 'int24', 'int24'], [
                                                 vault_ichi.address.checksum, limit_lower, limit_upper])

vault_pool.functions.positions(base_position_key).call(
), vault_pool.functions.positions(limit_position_key).call(),

# liquidity uint128, feeGrowthInside0LastX128 uint256, feeGrowthInside1LastX128 uint256, tokensOwed0 uint128, tokensOwed1 uint128


([3882924048014, 0, 0, 0, 0], [15481272409821, 0, 0, 0, 0])

In [ ]:
# fee
# all deposit
# all withdraw
# all re-balance => fee

display(pd.DataFrame(token0.fetch_events(token0.events.Transfer, from_block=0,
        argument_filters={'from': vault_ichi.address.checksum, 'to': affiliate.checksum})))
display(pd.DataFrame(token0.fetch_events(token0.events.Transfer, from_block=0,
        argument_filters={'from': vault_ichi.address.checksum, 'to': fee_recipient.checksum})))

# TODO: deposit / withdraw by proportion
# vault token value = (vault position value - fee owned by vault (10% of current in pool) / total Supply

# Flow:
# Deposit - Withdraw - Re-balance
# Deposit - Withdraw (In) / (vault position value - fee owned by vault (10% of current in pool)


,args,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,from,to,value
0,"(from, to, value)",Transfer,116,28,b'vD\xc79\xc9\xb3\x00\xbcF\xc0Hd\xaaC\xda_\xa7...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,"b'\x08lo}\xda(,\xb3\xa3*E\xe4\x0b\xfc\xdb\x9dN...",39652972,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,13836719961422953
1,"(from, to, value)",Transfer,23,5,b'C\xe7\xdb\xeav\x81>F\xd9%U\xd2\x8bJ+\xd2\xec...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,b'Q9\xc1\x17\xdb\x12\xcd\xac\xa9\x1c\xbe=\x8d ...,39669766,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,13887450278663709
2,"(from, to, value)",Transfer,132,34,b'f..\xbc\x10h\x12\x8b[\xd9\x11g\xfe\xc6\xb3\x...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,"b'\x96\xb0""\xd4-\xd4\x96J\x92\x074W@f\x1dr\xdd...",39703618,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,16350739173998050
3,"(from, to, value)",Transfer,88,25,b'F\xc2\xa6\x1f\xab\x82\xf3\xb7\xe2^SbY\x0e\x1...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,b'FHU\xd7\x91;\xa0\xb1@\xef0&\xd5\x99\x8d-\xe7...,39718182,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,22112048702363348
4,"(from, to, value)",Transfer,223,59,b'\x1c\x87\xa6\xe1\x94\xb1cE\xf5{\x80\x1c\xb63...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,"b""\x13\xd66\x8c{.\xaai\xfap{\\>\xcf\xf9\xf8\xa...",39729346,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,8004238651029347
...,...,...,...,...,...,...,...,...,...,...,...
136,"(from, to, value)",Transfer,50,11,b'\xc6DY\xb5db\\\xd4\r=\x94\xf4&yP\x8dk\x14\x0...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,b'\\3\xe8? \x1f\x1d\x0b\xaa\x87\xe2[\xc4}\xe7\...,41503095,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,565311745477418512
137,"(from, to, value)",Transfer,145,20,b'\x1e\xf6I\xddX\xebn\x8c\x99\x14\x7f\xff\xc0\...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,b'\xf6\x8di\xd1V9\xf5\xb55hg\x92\xda/\x00Hh\x0...,41504723,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,285062819897844433
138,"(from, to, value)",Transfer,74,18,b'\x8c\xdf\xfbY\x0cOH\xd6\xf5{pA\xf8\x00\x87P\...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,b'\xb6\xdd\xbf\xe80T\xbf\x044\xa7\x95\x96\xe5\...,41532362,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,1244329417727944758
139,"(from, to, value)",Transfer,401,69,b'\xe4\xea\x89\x89e\xee\x8bYy\xc8\xa2\x0e\xbc7...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,b'\xec\r\x94\xef\x06[\x04\xc0\xc6v\xd5\x9aZt\x...,41534042,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x3D1E6349f508199fDCAf339C1C351DfF1D5fd072,102462071144965941


,args,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,from,to,value
0,"(from, to, value)",Transfer,115,28,b'vD\xc79\xc9\xb3\x00\xbcF\xc0Hd\xaaC\xda_\xa7...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,"b'\x08lo}\xda(,\xb3\xa3*E\xe4\x0b\xfc\xdb\x9dN...",39652972,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,13836719961422952
1,"(from, to, value)",Transfer,22,5,b'C\xe7\xdb\xeav\x81>F\xd9%U\xd2\x8bJ+\xd2\xec...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,b'Q9\xc1\x17\xdb\x12\xcd\xac\xa9\x1c\xbe=\x8d ...,39669766,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,13887450278663709
2,"(from, to, value)",Transfer,131,34,b'f..\xbc\x10h\x12\x8b[\xd9\x11g\xfe\xc6\xb3\x...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,"b'\x96\xb0""\xd4-\xd4\x96J\x92\x074W@f\x1dr\xdd...",39703618,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,16350739173998050
3,"(from, to, value)",Transfer,87,25,b'F\xc2\xa6\x1f\xab\x82\xf3\xb7\xe2^SbY\x0e\x1...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,b'FHU\xd7\x91;\xa0\xb1@\xef0&\xd5\x99\x8d-\xe7...,39718182,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,22112048702363348
4,"(from, to, value)",Transfer,222,59,b'\x1c\x87\xa6\xe1\x94\xb1cE\xf5{\x80\x1c\xb63...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,"b""\x13\xd66\x8c{.\xaai\xfap{\\>\xcf\xf9\xf8\xa...",39729346,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,8004238651029346
...,...,...,...,...,...,...,...,...,...,...,...
136,"(from, to, value)",Transfer,49,11,b'\xc6DY\xb5db\\\xd4\r=\x94\xf4&yP\x8dk\x14\x0...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,b'\\3\xe8? \x1f\x1d\x0b\xaa\x87\xe2[\xc4}\xe7\...,41503095,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,565311745477418511
137,"(from, to, value)",Transfer,144,20,b'\x1e\xf6I\xddX\xebn\x8c\x99\x14\x7f\xff\xc0\...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,b'\xf6\x8di\xd1V9\xf5\xb55hg\x92\xda/\x00Hh\x0...,41504723,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,285062819897844432
138,"(from, to, value)",Transfer,73,18,b'\x8c\xdf\xfbY\x0cOH\xd6\xf5{pA\xf8\x00\x87P\...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,b'\xb6\xdd\xbf\xe80T\xbf\x044\xa7\x95\x96\xe5\...,41532362,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,1244329417727944758
139,"(from, to, value)",Transfer,400,69,b'\xe4\xea\x89\x89e\xee\x8bYy\xc8\xa2\x0e\xbc7...,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,b'\xec\r\x94\xef\x06[\x04\xc0\xc6v\xd5\x9aZt\x...,41534042,0x9ff3C1390300918B40714fD464A39699dDd9Fe00,0x0768A75F616B98ee0937673bD83B7aBF142236Ea,102462071144965941
